In [13]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

health_df=pd.read_csv('health2.csv')
print('단일 컬럼 데이터 추출:\n',health_df[  'DIS' ].head(3))
print('\n여러 컬럼들의 데이터 추출:\n', health_df[ ['SBP','DIS']].head(3))

X_features=health_df.iloc[:,:-1]
y_labels=health_df.iloc[:,-1]
print('피처 데이터 shape: {0}'.format(X_features.shape))
print(X_features)
print(y_labels)

단일 컬럼 데이터 추출:
 0    4
1    4
2    4
Name: DIS, dtype: int64

여러 컬럼들의 데이터 추출:
    SBP  DIS
0  116    4
1  100    4
2  100    4
피처 데이터 shape: (1000000, 7)
        SEX  age_arrange  BTH_G  SBP  DBP  FBS   BMI
0         1            1      1  116   78   94  16.6
1         1            1      1  100   60   79  22.3
2         1            1      1  100   60   87  21.9
3         1            1      1  111   70   72  20.2
4         1            1      1  120   80   98  20.0
...     ...          ...    ...  ...  ...  ...   ...
999995    2            6     27  120   70   81  23.1
999996    2            6     27  110   70  104  27.2
999997    2            6     27  115   53  110  25.2
999998    2            6     27  120   70   90  19.7
999999    2            6     27  116   73   92  17.6

[1000000 rows x 7 columns]
0         4
1         4
2         4
3         4
4         4
         ..
999995    2
999996    2
999997    1
999998    2
999999    4
Name: DIS, Length: 1000000, dtype: int64


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,r2_score, accuracy_score
from sklearn.tree import DecisionTreeClassifier

dt_clf=DecisionTreeClassifier()
X_train, X_test, y_train, y_test=train_test_split(X_features, y_labels, test_size=0.2, random_state=10)

dt_clf.fit(X_train, y_train)
pred=dt_clf.predict(X_test)
print('예측 정확도:{0:.4f}'.format(accuracy_score(y_test, pred)))

예측 정확도:0.6861


In [24]:
from lightgbm import LGBMClassifier
import pandas as pd

lgbm_wrapper=LGBMClassifier(n_estimaitors=400, random_state=10)

evals=[(X_train, y_train)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss',
                eval_set=evals, verbose=True)
preds=lgbm_wrapper.predict(X_test)
pred_proba=lgbm_wrapper.predict_proba(X_test)[:,1]


[LightGBM] [Warning] Unknown parameter: n_estimaitors
[1]	training's multi_logloss: 0.746317
Training until validation scores don't improve for 100 rounds
[2]	training's multi_logloss: 0.706783
[3]	training's multi_logloss: 0.677958
[4]	training's multi_logloss: 0.655896
[5]	training's multi_logloss: 0.638555
[6]	training's multi_logloss: 0.624635
[7]	training's multi_logloss: 0.613387
[8]	training's multi_logloss: 0.604141
[9]	training's multi_logloss: 0.596578
[10]	training's multi_logloss: 0.590309
[11]	training's multi_logloss: 0.585073
[12]	training's multi_logloss: 0.58068
[13]	training's multi_logloss: 0.576982
[14]	training's multi_logloss: 0.573882
[15]	training's multi_logloss: 0.571265
[16]	training's multi_logloss: 0.569038
[17]	training's multi_logloss: 0.567135
[18]	training's multi_logloss: 0.565517
[19]	training's multi_logloss: 0.564098
[20]	training's multi_logloss: 0.562884
[21]	training's multi_logloss: 0.561846
[22]	training's multi_logloss: 0.560948
[23]	training'

In [22]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score, precision_recall_curve, roc_curve

def get_clf_eval(y_test,pred=None, pred_proba=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test,pred)
    f1=f1_score(y_test, pred)
    roc_auc=roc_auc_score(y_test,pred_proba)
    
    print('오차행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f},재현율:{2:.4f}, F1:{3:.4f}, AUC:{4:.4f}'.format(accuracy,precision,recall,f1,roc_auc))
    

In [23]:
get_clf_eval(y_test, preds,pred_proba)

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].